# NOTE 
This notebook uses interactive examples - you'll need ipympl for the interactive plots to work. To install it: <br>
pip install ipympl <br>
There is also a 3 minute youtube video showing a real example which isn't included in this notebook. <br>
https://www.youtube.com/watch?v=fJ5YznNWPyE

In [1]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import matplotlib.gridspec as gridspec
import numpy as np
import math
%matplotlib widget

## Toy Example 1 
Consider the following piecewise ODE with a state dependent switching condition
$$\dot x = \begin{cases} p & x \leq 0 \\ -1 & x > 0 \end{cases} 
 \\ x(0) = 0
$$

Where $p > 0$ is a model parameter. For the continuous system, there is no solution - $x(t)$ is switching between regimes arbitrarily fast, so there is no function which can satisfy the ODE in a classical sense. 

We can still study the discrete system though -
$$x_{i+1} = \begin{cases} 
 x_i + p & x \leq 0 \\ 
  x_i - 1 & x > 0 \\ 
  \end{cases}$$
  
For this example, and the rest of the examples, we take the loss function $f(x)$ to be $\sum_i x_i$ for a discrete system and $\int_t x(t)$ for a continuous system.

The following code block will plot the system state $x_i$, the objective $f(x)$, and gradient $df/dp$. Try moving the slider to set the parameter value, and see how the objective/gradient are influenced.

In [6]:
timesteps = 20
prange = (.4,1.6)

def fun(x, p):
    return p if x <= 0 else -1

def solution(p, timesteps=timesteps):
    x = 0
    out = [x]
    for i in range(timesteps):
        x = x + fun(x, p)
        out.append(x)
    return out

def objective(out):
    return sum(out)

def obj_and_grad(p):
    obj = objective(solution(p))
    return obj, (objective(solution(p+1e-8))-obj)/1e-8

gs = gridspec.GridSpec(2,2)
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(gs[0,:])
ax2, ax3 = plt.subplot(gs[1,0]), plt.subplot(gs[1,1])
plt.subplots_adjust(bottom = .2)
x2 = np.linspace(prange[0], prange[1], 10000)
objlist = []
gradlist = []
for p in x2:
    obj, grad = obj_and_grad(p)
    objlist.append(obj)
    gradlist.append(grad)
obj, grad = obj_and_grad(.99)
x = list(range(timesteps))
out = solution(.99)
artist, = ax.plot(out)
ax.plot((0, timesteps), (0, 0))
artist2, = ax.plot(x, out[:-1], 'k.')
ax2.plot(x2, objlist, 'k.', markersize=2)
ax3.plot(x2, gradlist, 'k.', markersize=2)
artist3, = ax2.plot(.99, obj, 'r.')
artist4, = ax3.plot(.99, grad, 'r.')
ax2.set_ylabel('objective')
ax3.set_ylabel('gradient')
ax3.set_ylim([50, 150])
ax.set_ylim([-1, prange[1]+.1])
ax.set_ylabel('x(t)')
ax.set_xlabel('t')
ax.set_xticks(list(range(5,timesteps+1,5)))
ax.set_xlabel
axp = plt.axes([.15, 0.1, 0.65, 0.03])

def update(val):
    out = solution(val)
    artist.set_ydata(out)
    artist2.set_ydata(out[:-1])
    obj, grad = obj_and_grad(val)
    artist3.set_xdata(val)
    artist4.set_xdata(val)
    artist3.set_ydata(obj)
    artist4.set_ydata(grad)
    fig.canvas.draw_idle()

p_values = Slider(axp, 'p', prange[0], prange[1], valfmt='%.9f', valinit=.99)
p_values.on_changed(update)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

The black dots are the actual discrete times. The orange line is the switching condition; if the current timestep is below it, then we are in the first regime and move up by $p$. Otherwise, we move down by $-1$.

From the theorem, we know that on the interval $ p \in (.9, 1)$, the objective will be lipschitz continuous, because all the timesteps stay in the same regime. At $p=.9$, the timestep 20 flips regimes (i.e. it changes from up to down). This causes a discontinuity in both the gradient and objective, since both $h$ and its partial derivatives have discontinuities. At $p=1$, 10 timesteps (1, 3, 5, ... 19) flip regimes, resulting in a much larger discontinuity. Between $(.9, 1)$, even though the model is continuously switching between regimes (up, down, up, down), the objective is actually continuous, because each timestep stays in the same regime as long as $p \in (.9, 1)$.

Overall, we see a striated pattern, where we have many intervals where the objective is lipschitz continuous. At the ends of these intervals, we have values of $p$ which cause the switching times $\theta_j(p)$ to change. This striated pattern is canonical to discretized, state/parameter dependent piecewise DEs. An interesting observation is that the size of the discontinuity is proportional to the number of timesteps that flip. When only 1 timestep flips (e.g. at $p=.9$), the disconinuity is much smaller compared to when 2, 3, 6, or 10 timesteps flip (e.g. at $p = 3/4, 2/3, 1/2, 1$, respectively). 

You can tell from the objective that this system would be difficult to optimize with gradient based optimization. 

## Toy Example 2
$$ \ddot x = \begin{cases} 
 -x & x < p \\ 
  0 & x \geq p \\ 
  \end{cases}
 \\ x(0) = 0 \ \ \ \dot x(0) = 2
  $$
First, recall that if this didn't multiple regimes, and we just had $$\ddot x = -x$$, this is simply a simple harmonic oscillator, and $x(t)$ is just a sin wave. For this model, when we hit the switching condition $x(t) = p$, we freeze the acceleration, and keep moving with whatever speed we had.
Because the model is simple, in this case we can actually solve for the closed form solution. 
$$ x(t) = \begin{cases} 
 2 \sin(t) & t < t^* \\ 
  p + 2\cos(t^*) ( t - t^* ) & t \geq t^* \\
  \end{cases} \\
  \text{where } t^* = \arcsin(p/2)
$$
The code block below plots the continuous system.

In [3]:
prange = (.5, 2)
pinit = 1.99
def solution(p):
    if p > 2:
        x = np.linspace(0, 2*math.pi, 1000)
        y = 2*np.sin(x)
        return x, y
    point = math.asin(p/2)
    x1 = np.linspace(0, point, 1000)
    x2 = np.linspace(point, 2*math.pi, 1000)
    y1 = 2*np.sin(x1)
    y2 = p + 2*math.cos(point)*(x2 - point)
    x = np.append(x1, x2, axis=0)
    y = np.append(y1, y2, axis=0)
    return x, y

def obj_and_grad(p):
    if p <=2:
        star = math.asin(p/2)
        obj = 2 - 2*math.cos(star) +(2*math.pi-star)*(p-2*math.cos(star)*star)+math.cos(star)*(4*math.pi**2-star**2)
    else:
        obj = 0
    if p == 2:
        grad = math.nan
    elif p > 2:
        grad = 0
    else:
        ds = 1/2/(1-p**2/4)**.5
        grad = 2*math.sin(star)*ds + -ds*(p-2*math.cos(star)*star) + (2*math.pi-star)*(2*math.sin(star)*star*ds+1-2*math.cos(star)*ds) \
            -math.sin(star)*ds*(4*math.pi**2-star**2)+math.cos(star)*2*star*ds
    return obj, grad

gs = gridspec.GridSpec(2,2)
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(gs[0,:])
ax2, ax3 = plt.subplot(gs[1,0]), plt.subplot(gs[1,1])
plt.subplots_adjust(bottom = .2)
x2 = np.linspace(prange[0], prange[1], 10000)
objlist = []
gradlist = []
for p in x2:
    obj, grad = obj_and_grad(p)
    objlist.append(obj)
    gradlist.append(grad)
obj, grad = obj_and_grad(pinit)
x, out = solution(pinit)
artist, = ax.plot(x, out)
artist2, = ax.plot((0, 2*math.pi), (pinit, pinit), 'C0--', linewidth=1, alpha=.2)
ax2.plot(x2, objlist)
ax3.plot(x2, gradlist)
artist3, = ax2.plot(pinit, obj, 'r.')
artist4, = ax3.plot(pinit, grad, 'r.')
ax2.set_ylabel('objective')
ax3.set_ylabel('gradient')
ax.set_ylabel('x(t)')
ax.set_xlabel('t')
ax.set_ylim([-2,10])
ax3.set_ylim([-200,10])
axp = plt.axes([.15, 0.1, 0.65, 0.03])

def update(val):
    x, y = solution(val)
    artist.set_xdata(x)
    artist.set_ydata(y)
    artist2.set_ydata((val, val))
    obj, grad = obj_and_grad(val)
    artist3.set_xdata(val)
    artist4.set_xdata(val)
    artist3.set_ydata(obj)
    artist4.set_ydata(grad)
    fig.canvas.draw_idle()

p_values = Slider(axp, 'p', prange[0], prange[1]+.2, valfmt='%.9f', valinit=pinit)
p_values.on_changed(update)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

In this case, we know from the theorem that in the interval $p \in [0, 2-\epsilon]$ that the objective is lipschitz continuous. At $p=2$, the theorem is violated since all the times past $\pi /2$ flip regimes. Note that this is a special case, since we have a closed form solution which allows us to compute the switching times between regimes explicitly.

Now let's look at the discrete system.

In [4]:
prange = (.5, 2)
pinit = 1.99
timesteps = 100
x = np.linspace(0,2*math.pi, timesteps+1)

def fun(x, p):
    if x < p:
        return -x
    else:
        return 0

def solution2(p, dt=2*math.pi/timesteps):
    x = [0, 2]
    out = [x[0]]
    for i in range(timesteps):
         dx = fun(x[0], p)
         x[0] = x[0]+dt*x[1]
         x[1] = x[1] + dt*dx
         out.append(x[0])
    return out

def objective2(out, dt=2*math.pi/timesteps):
    return sum(out)*dt

def obj_and_grad2(p):
    obj = objective2(solution2(p))
    return obj, (objective2(solution2(p+1e-8))-obj)/1e-8

gs = gridspec.GridSpec(2,2)
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(gs[0,:])
ax2, ax3 = plt.subplot(gs[1,0]), plt.subplot(gs[1,1])
plt.subplots_adjust(bottom = .2)
x2 = np.linspace(prange[0], prange[1], 10000)
objlist = []
gradlist = []
for p in x2:
    obj, grad = obj_and_grad2(p)
    objlist.append(obj)
    gradlist.append(grad)
obj, grad = obj_and_grad2(pinit)
out = solution2(pinit)
artist1, = ax.plot(x, out)
artist5, = ax.plot((0, 2*math.pi), (pinit, pinit), 'C0--', alpha=.2)
artist22, = ax.plot(x, out, 'k.')
ax2.plot(x2, objlist, 'k.', markersize=2)
ax3.plot(x2, gradlist, 'k.', markersize=2)
artist33, = ax2.plot(pinit, obj, 'r.')
artist44, = ax3.plot(pinit, grad, 'r.')
ax.set_ylim([-2, 10])
ax2.set_ylabel('objective')
ax3.set_ylabel('gradient')
ax.set_ylabel('x(t)')
ax.set_xlabel('t')
ax.set_xlabel
axp2 = plt.axes([.15, 0.1, 0.65, 0.03])

def update(val):
    out = solution2(val)
    artist1.set_ydata(out)
    artist22.set_ydata(out)
    obj, grad = obj_and_grad2(val)
    artist33.set_xdata(val)
    artist44.set_xdata(val)
    artist33.set_ydata(obj)
    artist44.set_ydata(grad)
    artist5.set_ydata((val,val))
    fig.canvas.draw_idle()

p_values2 = Slider(axp2, 'p', prange[0], prange[1]+.2, valfmt='%.9f', valinit=pinit)
p_values2.on_changed(update)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

What happened to our gradient! Did we make a mistake? 
The problem is that for this system, the parameter $p$ only appears in the switching condition: neither of the two model regimes, $-x$ and $0$, depend on $p$ in any way. If you zoom in on the $x(t)$ plot around where the regime switches, and try changing the parameters, you can see this for yourself. If you change p, and the switching condition (the dashed line) stays between the discrete timesteps (the black dots), nothing changes! Everything stays exactly the same - meaning we have 0 gradient everywhere. In this case, we end up with an objective that looks like a floor function. 

This type of model can appear in practice, for example, for a lane changing model. The model outputs some discrete decisions, for example, to make a lane change or to stay in lane. In this case, the lane changing parameters might only appear in the switching conditions, meaning this type of model will have 0 gradient everywhere. An idea to solve this is to convert the model to output probabilities of changing lanes - this way the model output will have sensitivity with respect to the parameter.

## Example 3
The first two examples showed what can go wrong - now let's look at an example where gradient based optimization would work well.
$$ \dot x = \begin{cases} 
 -px & x > 2 \\ 
  -px - 2 & x \leq 2 \\ 
  \end{cases}$$
In this case, both regimes have sensitivity to the parameters, discontinuities are small, and the objective ends up being quasiconvex (however, we don't have lipschitz continuous, that's because the model has discontinuities between regimes. Meaning, $-px$ and $-px-2$ aren't equal when we switch regimes at $x=2$). 

In [5]:
prange = (0, 1)
pinit = .2
timesteps = 100
x = np.linspace(0,2, timesteps+1)

def fun(x, p):
    return -p*x if x > 2 else -p*x-2

def solution2(p, timesteps=timesteps, dt = 2/timesteps):
    x = 3
    out = [x]
    for i in range(timesteps):
        x = x + dt*fun(x, p)
        out.append(x)
    return out

def objective2(out):
    return sum(out)*2/timesteps

def obj_and_grad2(p):
    obj = objective2(solution2(p))
    return obj, (objective2(solution2(p+1e-8))-obj)/1e-8

gs = gridspec.GridSpec(2,2)
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(gs[0,:])
ax2, ax3 = plt.subplot(gs[1,0]), plt.subplot(gs[1,1])
plt.subplots_adjust(bottom = .2)
x2 = np.linspace(prange[0], prange[1], 10000)
objlist = []
gradlist = []
for p in x2:
    obj, grad = obj_and_grad2(p)
    objlist.append(obj)
    gradlist.append(grad)
obj, grad = obj_and_grad2(pinit)
out = solution2(pinit)
artist1, = ax.plot(x, out)
artist5, = ax.plot((0, 2), (2, 2), 'C0--', alpha=.2)
artist22, = ax.plot(x, out, 'k.')
ax2.plot(x2, objlist, 'k.', markersize=2)
ax3.plot(x2, gradlist, 'k.', markersize=2)
artist33, = ax2.plot(pinit, obj, 'r.')
artist44, = ax3.plot(pinit, grad, 'r.')
ax2.set_ylabel('objective')
ax3.set_ylabel('gradient')
ax.set_ylabel('x(t)')
ax.set_xlabel('t')
ax.set_xlabel
ax.set_ylim([-2, 3.2])
axp2 = plt.axes([.15, 0.1, 0.65, 0.03])

def update(val):
    out = solution2(val)
    artist1.set_ydata(out)
    artist22.set_ydata(out)
    obj, grad = obj_and_grad2(val)
    artist33.set_xdata(val)
    artist44.set_xdata(val)
    artist33.set_ydata(obj)
    artist44.set_ydata(grad)
    fig.canvas.draw_idle()

p_values2 = Slider(axp2, 'p', prange[0], prange[1]+.2, valfmt='%.9f', valinit=pinit)
p_values2.on_changed(update)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

## A real example (3 minute video)
https://www.youtube.com/watch?v=fJ5YznNWPyE


## Pre-print (accepted to transportation science)
https://arxiv.org/abs/1901.06452
